## Section 1: Historical Scenario Analysis

In [ ]:
import sys
sys.path.insert(0, '/Users/ajaiupadhyaya/Documents/Models')

from models.risk import (
    HistoricalScenarioAnalyzer,
    PortfolioStressTester,
    ScenarioAnalysisFull,
    SystemicRiskMeasures
)
import pandas as pd
import numpy as np
import yfinance as yf

# List available historical scenarios
scenarios = HistoricalScenarioAnalyzer.list_scenarios()
print(f"Available historical scenarios: {len(scenarios)}")
print("\nScenarios:")
for scenario in scenarios:
    print(f"  {scenario['name']} ({scenario['id']})")
    print(f"    {scenario['description']}")

## Section 2: Setup Portfolio for Stress Testing

In [ ]:
# Create sample portfolio allocation
portfolio = {
    'SPY': 350000,   # US Large Cap (35%)
    'QQQ': 200000,   # US Tech (20%)
    'IWM': 100000,   # US Small Cap (10%)
    'EEM': 150000,   # Emerging Markets (15%)
    'TLT': 150000,   # Bonds (15%)
    'GLD': 50000,    # Gold (5%)
}

total_value = sum(portfolio.values())

print("Portfolio Allocation:")
for asset, value in portfolio.items():
    pct = value / total_value * 100
    print(f"  {asset}: ${value:,.0f} ({pct:.1f}%)")
print(f"\nTotal Portfolio Value: ${total_value:,.0f}")

## Section 3: Apply Historical Scenarios

In [ ]:
# Initialize stress tester
stress_tester = PortfolioStressTester(portfolio)

# Run all historical scenarios
results = stress_tester.stress_test_historical()

# Rank by loss
ranked = stress_tester.rank_scenarios_by_loss(results)

print(f"\nPortfolio Stress Test Results ({len(ranked)} historical scenarios)")
print("\nTop 5 Worst Case Scenarios:")
print(f"{'Scenario':<35} {'Return':<12} {'Loss':<15}")
print("-" * 62)

for i, result in enumerate(ranked[:5]):
    ret_pct = result['portfolio_return'] * 100
    loss = result['portfolio_loss']
    print(f"{result['scenario']:<35} {ret_pct:>6.2f}% {loss:>14,.0f}")

## Section 4: Detailed Scenario Analysis

In [ ]:
# Get specific scenario
covid_scenario = HistoricalScenarioAnalyzer.get_scenario('covid_crash_2020')

print(f"Scenario: {covid_scenario.name}")
print(f"Description: {covid_scenario.description}")
print(f"\nAsset Shocks:")
print(f"{'Asset':<10} {'Shock':<12} {'Portfolio Weight':<18} {'Impact':<12}")
print("-" * 52)

# Apply to portfolio
result = stress_tester.apply_scenario(covid_scenario)

for asset, impact_data in result['asset_impacts'].items():
    shock = impact_data['shock'] * 100
    weight = impact_data['weight'] * 100
    impact = impact_data['impact'] * 100
    print(f"{asset:<10} {shock:>6.2f}% {weight:>8.1f}% {impact:>11.3f}%")

print(f"\nPortfolio Impact:")
print(f"  Return: {result['portfolio_return']*100:.2f}%")
print(f"  Loss: ${result['portfolio_loss']:,.0f}")
print(f"  Worst Position: {result['worst_position']}")
print(f"  Best Position: {result['best_position']}")

## Section 5: Summary Statistics from Stress Testing

In [ ]:
# Calculate summary statistics
stats = stress_tester.summary_statistics(results)

print("Summary Statistics Across All Scenarios:")
print(f"  Total Scenarios: {stats['total_scenarios']}")
print(f"  Mean Return: {stats['mean_return']*100:.3f}%")
print(f"  Std Dev: {stats['std_return']*100:.3f}%")
print(f"  Min Return: {stats['min_return']*100:.2f}%")
print(f"  Max Return: {stats['max_return']*100:.2f}%")
print(f"\n  Scenarios with Loss: {stats['scenarios_with_loss']}/{stats['total_scenarios']}")
print(f"  Mean Loss (when negative): {stats['mean_loss']*100:.3f}%")
print(f"  Worst Case Loss: ${stats['worst_case_loss']:,.0f}")
print(f"  Best Case Gain: ${stats['best_case_gain']:,.0f}")

## Section 6: Hypothetical Interest Rate Shock

In [ ]:
from models.risk import HypotheticalScenarioBuilder

# Create 50 bps rate shock scenario
rate_shock = HypotheticalScenarioBuilder.rate_shock_scenario(shock_bps=50)

print(f"Scenario: {rate_shock.name}")
print(f"\nAsset Impacts:")
print(f"{'Asset':<10} {'Shock':<12}")
print("-" * 22)

for asset, shock in sorted(rate_shock.shocks.items(), key=lambda x: x[1]):
    print(f"{asset:<10} {shock*100:>6.2f}%")

# Apply to portfolio
rate_result = stress_tester.apply_scenario(rate_shock)
print(f"\nPortfolio Impact: {rate_result['portfolio_return']*100:.3f}%")
print(f"Dollar Loss: ${rate_result['portfolio_loss']:,.0f}")

## Section 7: Credit Spread Widening Scenario

In [ ]:
# Create credit spread shock (100 bps widening)
credit_shock = HypotheticalScenarioBuilder.credit_spread_shock(shock_bps=100)

print(f"Scenario: {credit_shock.name}")
print(f"\nAsset Impacts:")

for asset, shock in sorted(credit_shock.shocks.items(), key=lambda x: x[1]):
    print(f"{asset:<10} {shock*100:>6.2f}%")

# Apply to portfolio
credit_result = stress_tester.apply_scenario(credit_shock)
print(f"\nPortfolio Impact: {credit_result['portfolio_return']*100:.3f}%")
print(f"Dollar Loss: ${credit_result['portfolio_loss']:,.0f}")

## Section 8: Equity Crash Scenario

In [ ]:
# Create 20% equity crash
equity_crash = HypotheticalScenarioBuilder.equity_crash_scenario(crash_pct=-0.20)

print(f"Scenario: {equity_crash.name}")

# Apply to portfolio
crash_result = stress_tester.apply_scenario(equity_crash)

print(f"\nImpacted Assets:")
print(f"{'Asset':<10} {'Weight':<12} {'Shock':<12} {'Impact':<12}")
print("-" * 46)

for asset, impact_data in crash_result['asset_impacts'].items():
    weight = impact_data['weight'] * 100
    shock = impact_data['shock'] * 100
    impact = impact_data['impact'] * 100
    print(f"{asset:<10} {weight:>6.1f}% {shock:>6.2f}% {impact:>11.3f}%")

print(f"\nPortfolio Impact: {crash_result['portfolio_return']*100:.3f}%")
print(f"Dollar Loss: ${crash_result['portfolio_loss']:,.0f}")

## Section 9: Currency Crisis Scenario

In [ ]:
# Create USD strength scenario
currency_crisis = HypotheticalScenarioBuilder.currency_crisis_scenario(usd_strength=0.10)

print(f"Scenario: {currency_crisis.name}")

# Apply to portfolio
currency_result = stress_tester.apply_scenario(currency_crisis)

print(f"\nPortfolio Impact: {currency_result['portfolio_return']*100:.3f}%")
print(f"Dollar Loss: ${currency_result['portfolio_loss']:,.0f}")

## Section 10: Tail Risk Analysis

In [ ]:
# Fetch historical returns for scenario analysis
print("Fetching historical data for analysis...")
tickers = list(portfolio.keys())
data = yf.download(tickers, period='2y', progress=False)['Adj Close']
returns = data.pct_change().dropna()

print(f"Data shape: {returns.shape}")
print(f"Period: {returns.index[0].date()} to {returns.index[-1].date()}")

# Tail risk analysis
scenario_analyzer = ScenarioAnalysisFull(returns)
tail_risks = scenario_analyzer.tail_risk_analysis(confidence_level=0.95)

print("\nTail Risk Analysis (5% tail):")
print(f"{'Asset':<10} {'Tail Mean':<12} {'Max Loss':<12} {'CVaR':<12}")
print("-" * 46)

for asset, metrics in tail_risks.items():
    tail_mean = metrics['tail_mean'] * 100
    max_loss = metrics['max_loss'] * 100
    cvar = metrics['expected_shortfall'] * 100
    print(f"{asset:<10} {tail_mean:>6.2f}% {max_loss:>11.2f}% {cvar:>11.2f}%")

## Section 11: Expected Shortfall & Coherent Risk

In [ ]:
# Expected Shortfall
es_metrics = scenario_analyzer.expected_shortfall(confidence_level=0.95)

print("Expected Shortfall (CVaR) Analysis:")
print(f"{'Asset':<10} {'VaR 95%':<12} {'CVaR 95%':<12} {'Tail Ratio':<12}")
print("-" * 46)

for asset, metrics in es_metrics.items():
    var = metrics['var'] * 100
    cvar = metrics['cvar'] * 100
    ratio = metrics['tail_ratio']
    print(f"{asset:<10} {var:>6.2f}% {cvar:>11.2f}% {ratio:>11.2f}")

# Coherent risk measures
coherent = scenario_analyzer.coherent_risk_measure()

print("\nCoherent Risk Measures:")
print(f"{'Asset':<10} {'Marginal VaR':<15} {'Severity-Weighted ES':<20}")
print("-" * 45)

for asset, metrics in coherent.items():
    mvr = metrics['marginal_var'] * 100
    smes = metrics['severity_weighted_es'] * 100
    print(f"{asset:<10} {mvr:>8.2f}% {smes:>18.2f}%")

## Section 12: Systemic Risk - Marginal Expected Shortfall

In [ ]:
# Systemic risk analysis
systemic_risk = SystemicRiskMeasures(returns)

# MES when SPY is in stress
print("Marginal Expected Shortfall (SPY stress):")
mes_results = systemic_risk.marginal_expected_shortfall('SPY', confidence_level=0.95)

print(f"{'Asset':<10} {'MES':<12} {'Stress Periods':<15}")
print("-" * 37)

for asset, metrics in mes_results.items():
    mes = metrics['mes'] * 100
    periods = metrics['stress_periods']
    print(f"{asset:<10} {mes:>6.2f}% {periods:>13} days")

print("\nSystemic Importance Ranking:")
systemic_scores = systemic_risk.systemic_importance()

for i, asset in enumerate(systemic_scores['ranking'][:5], 1):
    score = systemic_scores['scores'][asset]['systemic_score']
    impact = systemic_scores['scores'][asset]['avg_impact_on_others'] * 100
    print(f"{i}. {asset:<10} Score: {score:.4f} (Avg Impact: {impact:.3f}%)")

## Section 13: Conditional Value at Risk (CoVaR)

In [ ]:
# CoVaR analysis
covar_results = systemic_risk.conditional_value_at_risk('SPY', confidence_level=0.95)

print(f"CoVaR Analysis - When {covar_results['asset']} is stressed:")
print(f"\nAsset VaR: {covar_results['asset_var']*100:.2f}%")
print(f"Stress periods: {covar_results['stress_periods']}")

print(f"\nCoVaR for other assets:")
print(f"{'Asset':<10} {'CoVaR 95%':<12}")
print("-" * 22)

for asset, covar in sorted(covar_results['covar_estimates'].items(), key=lambda x: x[1]):
    print(f"{asset:<10} {covar*100:>6.2f}%")

print(f"\nPortfolio CoVaR: {covar_results['portfolio_covar']*100:.2f}%")

## Section 14: Correlation Breakdown in Stress

In [ ]:
# Correlation breakdown analysis
corr_breakdown = scenario_analyzer.correlation_breakdown(window=60, down_market_threshold=-0.01)

print("Correlation Breakdown Analysis:")

if corr_breakdown['correlation_increase']:
    print(f"\nCorrelation Changes (Normal vs Stress):")
    print(f"{'Pair':<20} {'Normal':<12} {'Stress':<12} {'Change':<12}")
    print("-" * 56)
    
    sorted_pairs = sorted(corr_breakdown['correlation_increase'].items(), 
                          key=lambda x: abs(x[1]), reverse=True)
    
    for pair, change in sorted_pairs[:5]:
        normal_corr = corr_breakdown['normal_correlation']
        stress_corr = corr_breakdown['stress_correlation']
        
        if normal_corr is not None and stress_corr is not None:
            assets = pair.split('-')
            i = corr_breakdown['normal_correlation'].columns.get_loc(assets[0])
            j = corr_breakdown['normal_correlation'].columns.get_loc(assets[1])
            
            normal = normal_corr.iloc[i, j]
            stress = stress_corr.iloc[i, j]
            
            print(f"{pair:<20} {normal:>6.2f} {stress:>11.2f} {change:>11.2f}")
else:
    print("Insufficient data for correlation breakdown analysis.")

## Section 15: Integrated Stress Test Report

In [ ]:
print("\n" + "="*70)
print("PORTFOLIO STRESS TEST SUMMARY REPORT")
print("="*70)

print(f"\nPortfolio Value: ${total_value:,.0f}")
print(f"\nHistorical Stress Test Results:")
print(f"  - Total scenarios tested: {stats['total_scenarios']}")
print(f"  - Scenarios with losses: {stats['scenarios_with_loss']}")
print(f"  - Worst case loss: ${stats['worst_case_loss']:,.0f}")
print(f"  - Average loss magnitude: ${stats['mean_loss']:,.0f}")

print(f"\nHypothetical Stress Scenarios:")
print(f"  - Interest rate shock (+50bps): ${rate_result['portfolio_loss']:,.0f}")
print(f"  - Credit spread widening (+100bps): ${credit_result['portfolio_loss']:,.0f}")
print(f"  - Equity crash (-20%): ${crash_result['portfolio_loss']:,.0f}")
print(f"  - USD strength (+10%): ${currency_result['portfolio_loss']:,.0f}")

print(f"\nTail Risk Metrics:")
print(f"  - Expected Shortfall (95%): {list(es_metrics.values())[0]['cvar']*100:.2f}% (avg)")

print(f"\nSystemic Risk:")
most_systemic = systemic_scores['ranking'][0]
print(f"  - Most systemically important asset: {most_systemic}")
print(f"  - Portfolio CoVaR (when SPY stressed): {covar_results['portfolio_covar']*100:.2f}%")

print("\n" + "="*70)